# <div style="text-align: center; background-color: #00FFFF	; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">📊 EDA |House Data | Visualization</div>

<a id="1"></a>
# <div style="text-align: center; background-color: #838B8B; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">1. Import Necessary Libraries</div>

In [ ]:
!pip install ydata-profiling

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import missingno as mno
import plotly.offline as pyo 
import plotly.figure_factory as ff
import plotly.io as pio
from wordcloud import WordCloud
color_pal = sns.color_palette()
plt.style.use('seaborn-dark-palette')
plt.style.use('dark_background')

import nltk

import warnings
warnings.filterwarnings('ignore')
sns.set_theme(style='darkgrid', palette='colorblind')
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()

#Model
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df=pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')
df.head()

<a id="1"></a>
# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">2. Exploratory Data Analysis 📊 </div>

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

In [ ]:
df.dtypes

In [ ]:
unique_values = df.nunique()
typpe_values = df.dtypes

summary_df = pd.DataFrame({'Unique Values': unique_values, 'Data Type': typpe_values})
print(summary_df.head(50))
print('======================')
print(summary_df.tail(31))

In [ ]:
df.info()

# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">3. Null values</div>

In [ ]:
print(df.isna().sum().head(50))
print('======================')
print(df.isna().sum().tail(31))

In [ ]:
plt.figure(figsize = (30,10))
df.isna().sum().plot(kind='bar')
plt.show()
plt.figure(figsize = (30,10))
sns.barplot(df.isna())
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Fill missing values in categorical columns with 'Unknown'
categorical_columns = [ 'FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']
df[categorical_columns] = df[categorical_columns].fillna('Unknown')

# Calculate mean and median for specific columns
mean_video_views_rank = df['LotFrontage'].mean()
median_Population = df['GarageYrBlt'].median()

# Fill missing values in specific columns with calculated values
df['LotFrontage'].fillna(mean_video_views_rank, inplace=True)
df['GarageYrBlt'].fillna(median_Population, inplace=True)

# Drop specific columns
df.drop(columns=['Alley','MiscFeature','Fence','PoolQC',], inplace=True)

# Drop rows with any remaining missing values
df = df.dropna()

# Display the shape of the DataFrame after changes
print(df.shape)

# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">4. Duplicate rows</div>

In [ ]:
# Finding duplicate rows
duplicate_rows = df[df.duplicated(keep='first')]

# Number of duplicate rows
num_duplicates = duplicate_rows.shape[0]

# Displaying the duplicate rows
print(f"Number of duplicate rows: {num_duplicates}")
duplicate_rows

# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">5. Data visualisation</div>

In [ ]:
# Columns for which you want to create plots
columns_to_plot = ['GarageType', 'SaleType','SaleCondition']

for column in columns_to_plot:
    top10_values = df[column].value_counts()[:10]
    
    # Create a bar plot using Plotly Express
    fig = px.bar(
        y=top10_values.values,
        x=top10_values.index,
        color=top10_values.index,
        color_discrete_sequence=px.colors.sequential.PuBuGn,
        text=top10_values.values,
        title=f'Top 10 {column}',
        template='plotly_white'
    )
    fig.update_layout(
        title_text=f'{column} Distribution',
        height=700,
        xaxis_title=column,
        yaxis_title="Count",
        font=dict(size=17, family="Franklin Gothic")
    )
    
    # Display the plot
    fig.show()


In [ ]:
# Columns and y-labels for which you want to create plots
columns_to_plot = [
    ('Exterior1st', 'SalePrice', 'SalePrice by Exterior1st'),
    ('GarageType', 'SalePrice', 'SalePrice by GarageType'),
    ('Neighborhood', 'SalePrice', 'SalePrice by Neighborhood')
]

for column, y_label, title in columns_to_plot:
    data = df.groupby(column)[y_label].sum().reset_index()
    
    # Create a line plot using Plotly Express
    fig = px.line(
        data,
        x=column,
        y=y_label,
        title=title,
        height=800
    )
    fig.update_layout(
        xaxis_title=column,
        yaxis_title=y_label,
        font=dict(size=17, family="Franklin Gothic")
    )
    
    # Display the plot
    fig.show()

In [ ]:

# Columns and y-labels for which you want to create plots
columns_to_plot = [
    ('Exterior2nd', 'Count   Exterior2nd by SalePrice', 'SalePrice'),
    ('Neighborhood', 'Count  Neighborhood by SalePrice', 'SalePrice')
]

# Initialize Plotly for notebook integration
pyo.init_notebook_mode(connected=True)

for column, title, y_label in columns_to_plot:
    # Calculate the average for the current column and sort
    average_data = df.groupby(column)[y_label].mean().reset_index()
    average_data = average_data.sort_values(by=y_label, ascending=False)
    
    # Select the top 10 items
    top_10_data = average_data.head(10)
    
    # Create a new figure using Plotly
    fig = go.Figure()
    
    # Add a bar trace to the figure
    fig.add_trace(go.Bar(
        x=top_10_data[column],
        y=top_10_data[y_label]
    ))
    
    # Update the layout and appearance of the plot
    fig.update_layout(
        title=title,
        xaxis_title=column,
        yaxis_title=y_label,
        template='plotly_white',
        font=dict(color='black'),
        height=800
    )
    
    # Display the plot
    pyo.iplot(fig)

In [ ]:
# Replace "df" with your actual DataFrame and "SaleCondition" and "SalePrice" with your actual column names
fig = px.box(df, x="SaleCondition", y="SalePrice", color="SaleType", notched=True)

fig.update_layout(
    title_text='Sale Price Distribution by Sale Condition',  # Update the title
    height=500,
    xaxis_title="Sale Condition",  # Update x-axis label
    yaxis_title="Sale Price",  # Update y-axis label
    font=dict(size=17, family="Franklin Gothic Book")  # Update font settings
)

fig.show()

In [ ]:
# Columns and titles for which you want to create sunburst charts
columns_to_plot = [
    ('SaleCondition', 'Distribution of SaleCondition'),
    ('SaleType', 'Distribution of SaleType'),
    ('LotShape','Distribution of LotShape')
]

for column, title in columns_to_plot:
    # Create a sunburst chart using Plotly Express
    fig = px.sunburst(df, path=[column], color_discrete_sequence=px.colors.qualitative.Set2)
    fig.update_layout(title_text=title, height=500)
    
    # Display the plot
    fig.show()

In [ ]:
# Create a scatter plot using Plotly Express
scatter_plot = px.scatter(
    df,
    x='Neighborhood',         # Data for the x-axis (neighborhoods)
    y='SalePrice',            # Data for the y-axis (sale prices)
    color='SaleCondition',    # Color code points based on sale condition
    # color_discrete_map={'Pave': 'red', 'Grvl': 'Blue'}  # Custom color mapping (optional)
)

# Update the layout of the scatter plot to include a dropdown menu
scatter_plot.update_layout(
    updatemenus=[
        dict(
            buttons=[
                dict(
                    args=[{'type': 'scatter'}],  # Set the plot type to scatter
                    label="Scatter Plot",       # Label for the first dropdown option
                    method="restyle"            # Restyle the plot with new settings
                ),
                dict(
                    args=[{'type': 'bar'}],     # Set the plot type to bar
                    label="Bar Chart",          # Label for the second dropdown option
                    method="restyle"            # Restyle the plot with new settings
                )
            ],
            direction="down",                 # Open the dropdown menu downward
            showactive=True,                  # Highlight the active button
        )
    ]
)

# Display the scatter plot with the dropdown menu
scatter_plot.show()


In [ ]:
fig = px.scatter(df, x="SalePrice", y="Neighborhood", size="SalePrice", color="SaleCondition",
                 hover_name="SaleType", log_x=True, size_max=60)
fig.show()

In [ ]:
fig = px.box(df, x="Neighborhood", y="SalePrice")
 
# showing the plot
fig.show()

In [ ]:
# Group by 'Sex' and 'Year', and calculate the sum of 'No of Suicides'
sex_year = df.groupby(["SaleType", "YrSold"])["SalePrice"].sum().reset_index()

# Create a bar plot for each type in 'Sex'
fig = px.bar(
    sex_year,
    x="YrSold",
    y="SalePrice",
    color="SaleType",
    facet_col="SaleType",
    labels={'YrSold': 'YrSold', 'SalePrice': 'SalePrice'},
    template='plotly',
    height=400
)

# Rotate x-axis labels for better readability
fig.update_xaxes(tickangle=90)

# Show the plots
fig.show()

In [ ]:
# Select the top 10 SalePrices in the DataFrame
top10_SalePrices = df['SalePrice'].sort_values(ascending=False).head(10)

# Create a bar plot using Plotly Express
# y-axis: SalePrice values, x-axis: Index of the top 10 SalePrices,
# color-coded by the index, and using a color palette
fig = px.bar(
    y=top10_SalePrices.values,      # y-values: top 10 SalePrices
    x=top10_SalePrices.index,       # x-values: indices of the top 10 SalePrices
    color=top10_SalePrices.index,   # Color the bars based on the indices
    color_discrete_sequence=px.colors.sequential.PuBuGn,  # Set color palette
    text=top10_SalePrices.values,   # Display the SalePrice values on top of the bars
    title='Top 10 SalePrices',       # Set the title of the plot
    template='plotly_dark'          # Use a dark template for the plot
)

# Update the layout of the plot
fig.update_layout(
    title_text='Top 10 SalePrice Distribution',  # Set the title of the plot
    height=700,                     # Set the height of the plot
    xaxis_title="Index",            # Label for the x-axis
    yaxis_title="Sale Price",       # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
def Freq_df(cleanwordlist):
    Freq_dist_nltk = nltk.FreqDist(cleanwordlist)
    df_freq = pd.DataFrame.from_dict(Freq_dist_nltk, orient='index')
    df_freq.columns = ['Frequency']
    df_freq.index.name = 'Term'
    df_freq = df_freq.sort_values(by=['Frequency'],ascending=False)
    df_freq = df_freq.reset_index()
    return df_freq

def Word_Cloud(data, color_background, colormap, title):
    plt.figure(figsize = (20,15))
    wc = WordCloud(width=1200, 
               height=600, 
               max_words=50,
               colormap= colormap,
               max_font_size = 100,
               random_state=88, 
               background_color=color_background).generate_from_frequencies(data)
    plt.imshow(wc, interpolation='bilinear')
    plt.title(title, fontsize=20)
    plt.axis('off')
    plt.show()

In [ ]:
freq_df = Freq_df(df['Neighborhood'].values.tolist())
data = dict(zip(freq_df['Term'].tolist(), freq_df['Frequency'].tolist()))
data = freq_df.set_index('Term').to_dict()['Frequency']

Word_Cloud(data ,'black','RdBu', 'WordCloud of Neighborhood')

In [ ]:
freq_df = Freq_df(df['Exterior2nd'].values.tolist())
data = dict(zip(freq_df['Term'].tolist(), freq_df['Frequency'].tolist()))
data = freq_df.set_index('Term').to_dict()['Frequency']

Word_Cloud(data ,'black','RdBu', 'WordCloud of Exterior2nd')

In [ ]:
# Calculate the average magnitude by month
magnitude_by_month = df.groupby('YearBuilt')['SalePrice'].mean().reset_index()

# Create a line plot using Plotly Express
fig = px.line(magnitude_by_month, x='YearBuilt', y='SalePrice',
              title='Average SalePrice by YearBuilt',
              labels={'SalePrice': 'SalePrice'},
              template='plotly_white')

# Display the plot
fig.show()

In [ ]:
# Calculate the average SalePrice by YearRemodAdd
magnitude_by_month = df.groupby('YearRemodAdd')['SalePrice'].mean().reset_index()

# Create a line plot using Plotly Express
fig = px.line(magnitude_by_month, x='YearRemodAdd', y='SalePrice',
              title='Average SalePrice by YearRemodAdd before RemodAdd',
              labels={'SalePrice': 'SalePrice'},
              template='plotly_white')

# Display the plot
fig.show()

In [ ]:
# Calculate the average SalePrice by LotArea
magnitude_by_month = df.groupby('LotArea')['SalePrice'].mean().reset_index()

# Create a line plot using Plotly Express
fig = px.line(magnitude_by_month, x='LotArea', y='SalePrice',
              title='Average SalePrice by LotArea',
              labels={'SalePrice': 'SalePrice'},
              template='plotly_white')

# Display the plot
fig.show()


# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">6. Categorical
</div>


In [ ]:
# Select columns with object (categorical) data types
num_cols = df.select_dtypes(include='object').columns.tolist()

# Initialize the LabelEncoder
le = LabelEncoder()

# Apply Label Encoding to the selected numerical columns
for x in num_cols:  
    df[x] = le.fit_transform(df[x])

# Now, your categorical columns (excluding column 0) have been converted to numerical values
df.head()


# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">7. Reduce dimensionality


</div>


In [ ]:
# Calculate the correlation matrix
Corr_Matrix = df.corr()

# Create a correlation heatmap using Plotly Express
fig = px.imshow(
    Corr_Matrix,  # Matrix containing the data
    labels=dict(x="Features", y="Features", color="Correlation"),  # Customize labels
    x=Corr_Matrix.columns,  # x-values: Features
    y=Corr_Matrix.columns,  # y-values: Features
    color_continuous_scale='blues',  # Set the color scale
    title='Correlation Heatmap',  # Set the title of the plot
    height=1000  # Set the height of the plot
)

# Display the plot
fig.show()

In [ ]:
print('Top 5 Most Positively Correlated to the To SalePrice')
Corr_Matrix['SalePrice'].sort_values(ascending=False).head(5)

In [ ]:
print('Top 5 Most Positively Correlated to SalePrice ')
Corr_Matrix['SalePrice'].sort_values(ascending=True).head(5)

In [ ]:
columns_to_drop = [col for col in Corr_Matrix.columns if abs(Corr_Matrix.loc['SalePrice', col]) < 0.3]
print('number of columns to drop ' ,len(columns_to_drop))

In [ ]:
df = df.drop(columns_to_drop, axis=1)
df.shape


# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">8. spliting the dataset



</div>


In [ ]:
X = df.drop(columns=["SalePrice"])
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Display the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">9. Model Building and Analysis




</div>

In [ ]:
models = {
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
}
best_model = None
best_r2 = 0

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    submit = pd.DataFrame()
    submit['SalePrice'] = y_test
    submit['SalePrice'] = y_pred
    submit = submit.reset_index()
    r2 = r2_score(y_test, y_pred)
    if r2 > best_r2:
        best_r2 = r2
        best_model = model.__class__.__name__

    print(f'{model_name}:')
    print(f'R2 Score: {r2:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
    print(submit.head(5))

    print('----------------------------------------')
print(f"The best performing model is: {best_model} with accuracy: {best_r2:.2f}")


# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">10. feature importances




</div>

In [ ]:
importances = model.feature_importances_
feature_names = X.columns
feature_importance_dict = dict(zip(feature_names, importances))
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

top_n = 5  # Set the number of top features to display
top_feature_names, top_importances = zip(*sorted_feature_importance[:top_n])

fig = px.bar(
    x=top_importances,
    y=top_feature_names,
    orientation='h',
    title='Top 5 Feature Importance',
    labels={'x': 'Importance', 'y': 'Feature'},
    color=top_importances,  # Color bars by importance values
    color_continuous_scale='reds',  # Choose a color scale
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')

fig.show()


# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">11. Feature selection
</div>

In [ ]:
# Assuming 'target_column' is the name of your target variable
X = df.drop(columns=['SalePrice'])
y = df['SalePrice']

# Scale the features to non-negative values using MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Select the top k features using f_regression
k = 5  # Number of features to select
selector = SelectKBest(score_func=f_regression, k=k)
X_selected = selector.fit_transform(X_scaled, y)

# Get the indices of the selected features
selected_indices = selector.get_support(indices=True)

# Create a new DataFrame with the selected features
selected_feature_names = [X.columns[i] for i in selected_indices]
X_new = pd.DataFrame(X_selected, columns=selected_feature_names)

# Combine the selected features with the target column
new_data = pd.concat([X_new, y], axis=1)

# Save the new dataset to a CSV file (optional)
new_data.to_csv('new_dataset_with_selected_features.csv', index=False)
new_data['SalePrice']=new_data['SalePrice'].fillna(13495.0)
new_data.head()


# <div style="text-align: center; background-color: #CDC8B1; font-family:Times New Roman; color: white; padding: 14px; line-height: 1;border-radius:20px">12. Model after Feature selection
</div>

In [ ]:
models = {
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
}
#====================================================================================
X = df.drop(columns=['SalePrice'])
y = df['SalePrice']

# Replace 'your_X_data' and 'your_y_data' with your actual feature and target data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#===============================================================================================
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    submit = pd.DataFrame()
    submit['SalePrice'] = y_test
    submit['SalePrice'] = y_pred
    submit = submit.reset_index()
    print(f'{model_name}:')
    print(f'R2 Score: {r2:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
    print(submit.head(5))

    print('----------------------------------------')
print(f"The best performing model is: {best_model} with accuracy: {best_r2:.2f}")